In [8]:
# Deleted old predictions
!rm -rf predict.txt predict.zip

In [9]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# Prediction

In [ ]:
import glob
import os
from ultralytics import YOLO
from tqdm import tqdm

# Đọc danh sách tệp ảnh
data_test = sorted(glob.glob("../data/raw/private test/*.jpg"))
batch_size = 64  # Đặt số lượng ảnh trong mỗi batch, điều chỉnh sao cho phù hợp với RAM của bạn

# Tạo model
model = YOLO("../models/base2/base/base.pt")

# Hàm chuyển đổi tọa độ từ xyxy sang xywh
def xyxy_to_xywh(xyxy, img_width, img_height):
    xmin, ymin, xmax, ymax = xyxy
    x_center = (xmin + xmax) / 2 / img_width
    y_center = (ymin + ymax) / 2 / img_height
    width = (xmax - xmin) / img_width
    height = (ymax - ymin) / img_height
    return x_center, y_center, width, height

# Xử lý theo batch và ghi kết quả
with open("predict.txt", "w") as f:
    for i in tqdm(range(0, len(data_test), batch_size)):
        batch_files = data_test[i:i + batch_size]
        results = model(batch_files)  # Dự đoán trên batch hiện tại
        from IPython.display import clear_output
        clear_output()

        for result in results:
            img_path = result.path
            img_name = os.path.basename(img_path)
            img_width, img_height = result.orig_img.shape[1], result.orig_img.shape[0]

            for box in result.boxes:
                xyxy = box.xyxy[0].cpu().numpy()
                class_id = int(box.cls[0].item())
                confidence = box.conf[0].item()
                x_center, y_center, width, height = xyxy_to_xywh(xyxy, img_width, img_height)

                # Ghi kết quả vào tệp
                f.write(f"{img_name} {class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {confidence:.6f}\n")
                

100%|██████████| 52/52 [06:05<00:00,  7.04s/it]


# Zip output

In [11]:
# Zip file predict.txt for submission
!zip -r predict.zip predict.txt

  adding: predict.txt (deflated 71%)
